In [1]:
# Imports
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [3]:
# Create connection connection, database, and collection within Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.atlstation_events_db
collection = db.event_list